In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#export
from nb_007 import *
import pandas as pd, re, spacy, html, os
from spacy.symbols import ORTH
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

# IMDB

## Data

### Reading the texts

In [ ]:
PATH = Path('data/aclImdb/')
CLAS_PATH = PATH/'clas'
LM_PATH = PATH/'lm'
os.makedirs(CLAS_PATH, exist_ok=True)
os.makedirs(LM_PATH, exist_ok=True)

In [ ]:
BOS,FLD,UNK,PAD = 'xxbos','xxfld','xxunk','xxpad'
TOK_UP,TK_REP,TK_WREP = 'xxup','xxrep','xxwrep'

In [ ]:
CLASSES = ['neg', 'pos', 'unsup']

def get_texts(path):
    texts,labels = [],[]
    for idx,label in enumerate(CLASSES):
        for fname in (path/label).glob('*.*'):
            texts.append(fname.open('r', encoding='utf8').read())
            labels.append(idx)
    return np.array(texts),np.array(labels)

train_texts,train_labels = get_texts(PATH/'train')
valid_texts,valid_labels = get_texts(PATH/'test')

In [ ]:
train_idx = np.random.permutation(len(train_texts))
valid_idx = np.random.permutation(len(valid_texts))

In [ ]:
train_texts,train_labels = train_texts[train_idx],train_labels[train_idx]
valid_texts,valid_labels = valid_texts[valid_idx],valid_labels[valid_idx]

In [ ]:
train_df = pd.DataFrame({'text':train_texts, 'labels':train_labels}, columns=['labels','text'])
valid_df = pd.DataFrame({'text':valid_texts, 'labels':valid_labels}, columns=['labels','text'])

We put aside the unsup labels for the classification but keep them to finetune the language model.

In [ ]:
train_df[train_df['labels']!=2].to_csv(CLAS_PATH/'train.csv', header=False, index=False)
valid_df.to_csv(CLAS_PATH/'valid.csv', header=False, index=False)

In [ ]:
all_texts = np.concatenate([train_texts,valid_texts])
idx = np.random.permutation(len(all_texts))
cut = int(0.1 * len(idx))
train_df = pd.DataFrame({'text':all_texts[idx[cut:]], 'labels':[0] * (len(all_texts)-cut)}, columns=['labels','text'])
valid_df = pd.DataFrame({'text':all_texts[idx[:cut]], 'labels':[0] * cut}, columns=['labels','text'])

In [ ]:
train_df.to_csv(LM_PATH/'train.csv', header=False, index=False)
valid_df.to_csv(LM_PATH/'valid.csv', header=False, index=False)

### Tokenization + Numericalization

In [ ]:
def partition(a, sz): 
    """splits iterables a in equal parts of size sz"""
    return [a[i:i+sz] for i in range(0, len(a), sz)]

def partition_by_cores(a, n_cpus):
    return partition(a, len(a)//n_cpus + 1)

def num_cpus():
    try:
        return len(os.sched_getaffinity(0))
    except AttributeError:
        return os.cpu_count()

In [ ]:
class SpacyTokenizer():
    "Little wrapper around a spacy tokenizer"
    
    def __init__(self, lang):
        self.tok = spacy.load(lang)
    
    def tokenizer(self, t):
        return [t.text for t in self.tok.tokenizer(t)]
    
    def add_special_cases(self, toks):
        for w in toks:
            self.tok.tokenizer.add_special_case(w, [{ORTH: w}])

In [ ]:
train_df = pd.read_csv(LM_PATH/'train.csv', header=None, chunksize=10)
trn_df = next(train_df)
test_tok = SpacyTokenizer('en')
test_txt = trn_df.iloc[0][1]
test_tok.tokenizer(test_txt)

In [ ]:
class Tokenizer():
    def __init__(self, tok_fn=SpacyTokenizer, lang:str='en', rules:Collection[Callable[[str],str]]=None, 
                 special_cases:Collection[str]=None, n_cpus = None):
        self.tok_fn,self.lang,self.special_cases = tok_fn,lang,special_cases
        self.rules = rules if rules else []
        for rule in self.rules:
            if hasattr(rule, 'compile'): rule.compile()
        self.n_cpus = n_cpus or num_cpus()//2
    
    def __repr__(self):
        res = f'Tokenizer {self.tok_fn.__name__} in {self.lang} with the following rules:\n'
        for rule in self.rules: res += f' - {rule.__name__}\n'
        return res
    
    def proc_text(self, t, tok):
        for rule in self.rules: t = rule(t)
        return tok.tokenizer(t)
    
    def process_all_1thread(self, texts):
        tok = self.tok_fn(self.lang)
        if self.special_cases: tok.add_special_cases(self.special_cases)
        return [self.proc_text(t, tok) for t in texts]

    def process_all(self, texts):
        if self.n_cpus <= 1: return self.process_all_1thread(texts)
        with ProcessPoolExecutor(self.n_cpus) as e:
            return sum(e.map(self.process_all_1thread, partition_by_cores(texts, self.n_cpus)), [])

In [ ]:
def sub_br(t):
    "Replaces the <br /> by \n"
    re_br = re.compile(r'<\s*br\s*/?>', re.IGNORECASE)
    return re_br.sub("\n", t)

def spec_add_spaces(t):
    return re.sub(r'([/#])', r' \1 ', t)

def rm_useless_spaces(t):
    return re.sub(' {2,}', ' ', t)

def replace_rep(t):
    def _replace_rep(m):
        c,cc = m.groups()
        return f' {TK_REP} {len(cc)+1} {c} '
    re_rep = re.compile(r'(\S)(\1{3,})')
    return re_rep.sub(_replace_rep, t)
    
def replace_wrep(t):
    def _replace_wrep(m):
        c,cc = m.groups()
        return f' {TK_WREP} {len(cc.split())+1} {c} '
    re_wrep = re.compile(r'(\b\w+\W+)(\1{3,})')
    return re_wrep.sub(_replace_wrep, t)

def deal_caps(t):
    res = []
    for s in re.findall(r'\w+|\W+', t):
        res += ([TOK_UP,s.lower()] if (s.isupper() and (len(s)>2)) else [s.lower()])
    return ' '.join(res)

def fixup(x):
    re1 = re.compile(r'  +')
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>',UNK).replace(' @.@ ','.').replace(
        ' @-@ ','-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x))
    
rules = [sub_br, spec_add_spaces, rm_useless_spaces, replace_rep, replace_wrep, deal_caps, fixup]

In [ ]:
tokenizer = Tokenizer(rules=rules, special_cases=[BOS, FLD, 'xxunk', 'xxpad'], n_cpus=1)

In [ ]:
tokenizer

In [ ]:
assert sub_br('end <br /><br /> begins again')=='end \n\n begins again'

In [ ]:
assert spec_add_spaces('\#%') == '\\ # %'

In [ ]:
assert rm_useless_spaces('this             is') == 'this is'

In [ ]:
assert replace_rep('ffffffive .') == ' xxrep 6 f ive .'

In [ ]:
assert replace_wrep('five five five five .') == ' xxwrep 4 five  .'

In [ ]:
assert deal_caps('ANGRY') == 'xxup angry'

In [ ]:
def get_chunk_length(csv_name, chunksize):
    dfs = pd.read_csv(csv_name, header=None, chunksize=chunksize)
    l = 0
    for _ in dfs: l+=1
    return l

def get_total_length(csv_name, chunksize):
    dfs = pd.read_csv(csv_name, header=None, chunksize=chunksize)
    l = 0
    for df in dfs: l+=len(df)
    return l

In [ ]:
def maybe_copy(old_fname, new_fname):
        if not os.path.isfile(new_fname) or os.path.getmtime(new_fname) < os.path.getmtime(old_fname):
            shutil.copyfile(old_fname, new_fname)

In [ ]:
TextMtd = IntEnum('TextMtd', 'CSV TOK IDS')

import shutil

class TextDataset():
    "Put a train.csv and valid.csv files in a folder and this will take care of the rest."
    
    def __init__(self, path, tokenizer, max_vocab=30000, chunksize=10000, train_name='train', valid_name='valid',
                 min_freq=2, n_labels=1, create_mtd=TextMtd.CSV):
        self.path,self.tokenizer,self.max_vocab,self.min_freq = Path(path/'tmp'),tokenizer,max_vocab,min_freq
        self.chunksize,self.train_name,self.valid_name,self.n_labels = chunksize,train_name,valid_name,n_labels
        self.create_mtd = create_mtd
        os.makedirs(self.path, exist_ok=True)
        if not self.check_toks(): self.tokenize()
        if not self.check_ids():  self.numericalize()
        
        self.itos = pickle.load(open(self.path/'itos.pkl', 'rb'))
        self.train_ids = np.load(self.path/'train_ids.npy')
        self.valid_ids = np.load(self.path/'valid_ids.npy')
        self.train_lbl = np.load(self.path/'train_lbl.npy')
        self.valid_lbl = np.load(self.path/'valid_lbl.npy')
    
    def check_ids(self):
        if self.create_mtd >= TextMtd.IDS: return True
        if not np.all([os.path.isfile(fname) for fname in self.id_files]): return False
        itos = pickle.load(open(self.path/'itos.pkl', 'rb'))
        with open(self.path/'numericalize.log') as f:
            if len(itos) != int(f.read()) or len(itos) > self.max_vocab + 2: return False
        for tok_file,id_file in zip(self.tok_files[:-1], self.id_files[:-2]):
            if os.path.getmtime(tok_file) > os.path.getmtime(id_file): return False
            toks,ids = np.load(tok_file),np.load(id_file)
            if len(toks) != len(ids): return False
        return True
        
    def check_toks(self):
        if self.create_mtd >= TextMtd.TOK: return True
        if not np.all([os.path.isfile(fname) for fname in self.tok_files]): return False
        with open(self.path/'tokenize.log') as f:
            if repr(self.tokenizer) != f.read(): return False
        for csv_file,tok_file in zip(self.csv_files, self.tok_files[:-1]):
            if os.path.getmtime(csv_file) > os.path.getmtime(tok_file): return False
        return True
    
    def tokenize(self):
        print('Tokenizing the texts. This might take a while so you should grab a coffee.')
        for name in [self.train_name, self.valid_name]:
            print(f'Tokenizing {name}')
            curr_len = get_chunk_length(self.path/f'{name}.csv', self.chunksize)
            dfs = pd.read_csv(self.path/f'{name}.csv', header=None, chunksize=self.chunksize)
            tokens,labels = [],[]
            for _ in progress_bar(range(curr_len), leave=False):
                df = next(dfs)
                lbls = df.iloc[:,range(self.n_labels)].values.astype(np.int64)
                texts = f'\n{BOS} {FLD} 1 ' + df[self.n_labels].astype(str)
                for i in range(self.n_labels+1, len(df.columns)): 
                    texts += f' {FLD} {i-n_lbls} ' + df[i].astype(str)
                toks = self.tokenizer.process_all(texts)
                tokens += toks
                labels += labels
            np.save(self.path/f'{name}_tok.npy', np.array(tokens))
            np.save(self.path/f'{name}_lbl.npy', np.array(labels))
        with open(self.path/'tokenize.log','w') as f: f.write(repr(self.tokenizer))
        
    def numericalize(self):
        print('Changing tokens to numbers.')
        train_tokens = np.load(self.path/f'{self.train_name}_tok.npy')
        freq = Counter(p for o in train_tokens for p in o)
        itos = [o for o,c in freq.most_common(self.max_vocab) if c > self.min_freq]
        itos.insert(0, 'xxpad')
        itos.insert(0, 'xxunk')
        stoi = collections.defaultdict(lambda:0, {v:k for k,v in enumerate(itos)})
        pickle.dump(itos, open(self.path/'itos.pkl', 'wb'))
        with open(self.path/'numericalize.log','w') as f: f.write(str(len(itos)))
        for name in [self.train_name, self.valid_name]:
            toks = np.load(self.path/f'{name}_tok.npy')
            ids = np.array([([stoi[w] for w in s]) for s in toks])
            np.save(self.path/f'{name}_ids.npy', ids)
    
    def clear(self):
        files = [self.path/f'{name}_ids.npy' for name in [self.train_name,self.valid_name]] 
        files += [self.path/'itos.pkl']
        files += [self.path/f'{name}_tok.npy' for name in [self.train_name,self.valid_name]]
        for file in files:
            if os.path.isfile(file): os.remove(file)
    
    @property
    def csv_files(self): return [self.path/f'{name}.csv' for name in [self.train_name,self.valid_name]]
    @property
    def tok_files(self): 
        tok_files = [self.path/f'{name}_tok.npy' for name in [self.train_name,self.valid_name]]
        return tok_files + [self.path/'tokenize.log']
    @property
    def id_files(self):
        id_files = [self.path/f'{name}_ids.npy' for name in [self.train_name,self.valid_name]] 
        return id_files + [self.path/'itos.pkl', self.path/'numericalize.log']
    
    @classmethod
    def from_ids(cls, folder, train_ids='train_tok.npy', valid_ids='valid_tok.npy', 
                 train_lbl='train_lbl.npy', valid_lbl='train_lbl.npy', **kwargs):
        train_name,valid_name = train_ids[:-8],valid_ids[:-8]
        for ids, name in zip([train_ids,valid_ids], [train_name,valid_name]):
            maybe_copy(Path(folder)/ids, Path(folder)/'tmp'/f'{name}_ids.npy')
        for lbl, name in zip([train_lbl,valid_lbl], [train_name,valid_name]):
            maybe_copy(Path(folder)/lbl, Path(folder)/'tmp'/f'{name}_lbl.npy')
        return cls(folder, None, train_name=train_name, valid_name=valid_name, create_mtd=TextMtd.IDS, **kwargs)
    
    @classmethod
    def from_tokens(cls, folder, train_tok='train_tok.npy', valid_tok='valid_tok.npy', 
                    train_lbl='train_lbl.npy', valid_lbl='train_lbl.npy', **kwargs):
        train_name,valid_name = train_tok[:-8],valid_tok[:-8]
        for tok, name in zip([train_tok,valid_tok], [train_name,valid_name]):
            maybe_copy(Path(folder)/tok, Path(folder)/'tmp'/f'{name}_tok.npy')
        for lbl, name in zip([train_lbl,valid_lbl], [train_name,valid_name]):
            maybe_copy(Path(folder)/lbl, Path(folder)/'tmp'/f'{name}_lbl.npy')
        return cls(folder, None, train_name=train_name, valid_name=valid_name, create_mtd=TextMtd.TOK, **kwargs)
    
    @classmethod
    def from_csv(cls, folder, tokenizer, train_csv='train.csv', valid_csv='valid.csv', **kwargs):
        train_name,valid_name = train_csv[:-4],valid_csv[:-4]
        for csv, name in zip([train_csv,valid_csv], [train_name,valid_name]):
            maybe_copy(Path(folder)/csv, Path(folder)/'tmp'/f'{name}.csv')
        return cls(folder, tokenizer, train_name=train_name, valid_name=valid_name, **kwargs)
    
    @classmethod
    def from_folder(cls, folder, tokenizer, classes=None, train_name='train', valid_name='valid', **kwargs):
        path = Path(folder)/'tmp'
        os.makedirs(path, exist_ok=True)
        if classes is None: classes = [cls.name for cls in find_classes(Path(folder/train_name))]
        for name in [train_name, valid_name]:
            texts,labels = [],[]
            for idx,label in enumerate(classes):
                for fname in (Path(folder)/name/label).glob('*.*'):
                    texts.append(fname.open('r', encoding='utf8').read())
                    labels.append(idx)
            texts,labels = np.array(texts),np.array(labels)
            idx = np.random.permutation(len(texts))
            texts,labels = texts[idx],labels[idx]
            df = pd.DataFrame({'text':texts, 'labels':labels}, columns=['labels','text'])
            if os.path.isfile(path/f'{name}.csv'):
                if get_total_length(path/f'{name}.csv', 10000) != len(df):
                    df.to_csv(path/f'{name}.csv', index=False, header=False)
            else: df.to_csv(path/f'{name}.csv', index=False, header=False)
        return cls(folder, tokenizer, train_name=train_name, valid_name=valid_name, **kwargs)

In [ ]:
tokenizer = Tokenizer(rules=rules, special_cases=[BOS, FLD, 'xxunk', 'xxpad'], n_cpus=1)

In [ ]:
data = TextDataset.from_csv(LM_PATH, tokenizer, chunksize=1000)

In [ ]:
data = TextDataset.from_ids(LM_PATH)

In [ ]:
#data.clear()

In [ ]:
def make_sample(dir_name='tst_folders'):
    os.makedirs(PATH/dir_name, exist_ok=True)
    PATH1 = PATH/dir_name
    for name,name1 in zip(['train', 'valid'],['train', 'test']):
        os.makedirs(PATH1/name, exist_ok=True)
        for clas in ['neg', 'pos']:
            os.makedirs(PATH1/name/clas, exist_ok=True)
            fnames = list((PATH/name1/clas).glob('*.txt'))
            for i in range(2000):
                shutil.copy(fnames[i], PATH1/name/clas/fnames[i].name)

In [ ]:
#make_sample()

In [ ]:
data = TextDataset.from_folder(PATH/'tst_folders', tokenizer, chunksize=1000)